In [1]:
import sys  
sys.path.insert(0, '/home/iftekhar/myworkplace/AI-system/retrieval_Model/Page_Ranking_Experiment/methods_collection/')
import ranking_help_methods 
import make_question as question_maker
import vectorization
import sequence_handler
import corpus_handling_methods
import pandas as pd
import ranking
import re
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
vectorizer = CountVectorizer()
TFIDF_vectorizer = TfidfVectorizer()

In [2]:
dataset = pd.read_csv("../../../Helpers/Title_link_merged_corpus.csv")
dataset = dataset.iloc[:,2:]
dataset = dataset.rename(columns={"Article": "Data"})
dataset.Data = dataset.Data.apply(lambda x: corpus_handling_methods.clean_text(x))
dataset.head()

,Data,Summary,PageID
0,カスタマイザー から サブタイトル 設定 MobiControl 設定 順序 MobiCon...,MobiControl v14 Manual – MobiControl v14 Manual,0
1,MobiControl v Manual Apple 製品 MobiControl 管理 i...,Apple製品 – MobiControl v14 Manual ios overall,1
2,MobiControl v Manual Android 設定 モード A MobiCont...,Android端末 – MobiControl v14 Manual plus overall,2
3,MobiControl v Manual Windows PC 根本 セキュリティ 対策 リ...,Windows PC – MobiControl v14 Manual wpc overall,3
4,MobiControl v Manual Windows Embedded 端末 Windo...,Windows Embedded – MobiControl v14 Manual wm o...,4


In [3]:
split_corpus = corpus_handling_methods.corpus_split(dataset, 8)
split_corpus.head()

,Data,PageID
0,カスタマイザー から サブタイトル 設定 MobiControl 設定 順序 MobiCon...,0
1,設定 順序 MobiControl 設定 順序 行い 端末 OS,0
2,対象 オンプレミス MobiControl サーバ のみ 必要 な 設定,0
3,事項 iOS 端末 macOS コンピュータ 必須 接続 プロファイル,0
4,作成 ADDS Active Directory Domain Service サーバ 間,0


In [4]:
perpage_dataset = corpus_handling_methods.corpus_per_page(split_corpus)
perpage_dataset.head() 

,Data,PageID
0,カスタマイザー から サブタイトル 設定 MobiControl 設定 順序 MobiCon...,0
1,MobiControl v Manual Apple 製品 MobiControl 管理 i...,1
2,MobiControl v Manual Android 設定 モード A MobiCont...,2
3,MobiControl v Manual Windows PC 根本 セキュリティ 対策 リ...,3
4,MobiControl v Manual Windows Embedded 端末 Windo...,4


In [8]:
sample_size = 100
questions_samples = question_maker.question_dataframe_generator_1000(split_corpus, sample_size)
questions_samples.head()

,Question,PageID
0,行 アプリ 多い CSV 型式 アプリ リスト どんな,282
1,により ファイル 同期 終了 た 新しい 端末 作り方は,132
2,れ なく アプリ 起動 なく プロファイル 端末 どうでしたか,40
3,抜く 可能 性 そうすると リモート から WIPE 使い始める方法を教えてください,258
4,入れる 検索 表示 ページ 訳す リンク 表示 使い始めるには,185


In [6]:
vec = vectorization.vector_fit(TFIDF_vectorizer, split_corpus)

In [10]:
sample_count = 0
sum_score = 0
container = []
extra_ranks = 0
for index, col in questions_samples.iterrows():
    print(col['Question'], col['PageID'])
    query = str(col['Question'])
    question_parts = question_maker.making_query_collection(query)
    collector = sequence_handler.sequence_searcher(perpage_dataset, question_parts)
    perpage_sequence_match = sequence_handler.perpage_sequence_match(collector, 
                                        perpage_dataset, split_corpus, query)
    ranks = ranking.crude_ranks(perpage_sequence_match, query, vec)
    filtered_ranks = ranking.filtering_ranks(ranks, perpage_sequence_match, query, vec)
    print(col['PageID'], "<=>", filtered_ranks)
    
#     if filtered_ranks[0][1] < 0.75:
#         print("Calling Pretrained Model")
        
    page_answers = []
    prediction_scores = []
    for ids, score in filtered_ranks:
        page_answers.append(ids)
        prediction_scores.append(score)
    MRR_score = ranking.mean_reciprocal_rank_score(col['PageID'], page_answers)
    sum_score += MRR_score
    container.append([MRR_score, col['PageID'], page_answers, prediction_scores, col['Question']])
    sample_count += 1
    
result = pd.DataFrame(container, columns=['score', 'actual_answer',
'page_answers', 'prediction_scores', 'query'])
result.to_csv('seq_matcher_TFIDF_Vectorizer_performance.csv')
score = sum_score/sample_count
score



行 アプリ 多い CSV 型式 アプリ リスト どんな 282
282 <=> [[282, 0.7923927570081397], [168, 0.546304349693667], [206, 0.4227681508833792]]
により ファイル 同期 終了 た 新しい 端末 作り方は 132
132 <=> [[132, 0.870791548860434], [86, 0.5829938999260449], [413, 0.5117636559418992]]
れ なく アプリ 起動 なく プロファイル 端末 どうでしたか 40
40 <=> [[40, 0.8953556506745068], [156, 0.5392820916053557], [252, 0.5192435151913505]]
抜く 可能 性 そうすると リモート から WIPE 使い始める方法を教えてください 258
258 <=> [[138, 0.5555571622797968], [363, 0.4405827062090858], [152, 0.32719555522775856]]
入れる 検索 表示 ページ 訳す リンク 表示 使い始めるには 185
185 <=> [[185, 0.9297084381172427], [292, 0.4486762330093957], [15, 0.3968469927858153]]
アプリケーション 指定 アプリ ファイル パス 記述 項目 使い始める方法を教えてください 350
350 <=> [[317, 0.8871279805249745], [350, 0.8871279805249745], [317, 0.6131107646127892]]
サーバ ID など 盗ま れ ない な どんな 3
3 <=> [[3, 0.7277138249223319], [75, 0.37098123095113494], [6, 0.29338170181715373]]
設定 画面 アカウント ユーザ Exchange アカウント ユーザー どうですか 211
211 <=> [[117, 0.9878806883214886], [163, 0.9878806883214886], [211, 0.9878

6 <=> [[30, 0.6360456519099758], [53, 0.505725420602684], [72, 0.505725420602684]]
収集 データ 種類 指定 た 監視 データ 作り方は 131
131 <=> [[131, 0.9757390857679615], [132, 0.8202368914151577], [248, 0.8056224070912137]]
パスワード パスワード なけれ ば ブランク デバイス ID どの 338
338 <=> [[338, 0.9006328715728352], [308, 0.8269031065474022], [280, 0.6499221283696973]]
プライマリ SMTP ダウン た 代替 SMTP サーバ 開発を開始する場所 64
64 <=> [[64, 0.9876238296761979], [65, 0.9876238296761979], [64, 0.6812176819393279]]
プロファイル 項目 削除 F 作成 済 構成 作りたい 53
53 <=> [[53, 0.9487340380257836], [105, 0.9487340380257836], [396, 0.8325222880093235]]
まで 時間 始める方法 405
405 <=> [[151, 0.6768972233937595], [315, 0.5221334627185441], [346, 0.5221334627185441]]
カード 抜か WiFi 経由 接続 運用 いる 何ができますか 29
29 <=> [[29, 1.0000000000000002], [372, 0.6058707853126855], [411, 0.5503186557856355]]
現れる メニュー DEP 指定 管理 項目 クリック 何ができますか 387
387 <=> [[387, 0.9599691237595577], [367, 0.5506632949267265], [134, 0.45073673773693046]]
構成 プロファイル 作成 終わる OK ボタン 押し 何が 312
312 <=> [[179, 1.0], [196, 1

0.7515686274509804

In [14]:
filtered_ranks[0][1]

0.975616546550444

In [15]:
filtered_ranks

[[24, 0.975616546550444], [443, 0.975616546550444], [128, 0.4409370517422057]]

In [41]:
[(406, array([0.39501141])), (412, array([0.39501141])), (406, array([0.37375799]))]

[[2, ['団体 組織 そっ 階層 端末 グループ 作成 組織']],
 [3,
  ['コンソール 操作 従来 端末 グループ 戻し 指定 アプリ',
   '移動 隔離 実施 異常 端末 グループ 構成 プロファイル',
   'PC 特別 端末 グループ 自動 移動 せ プロファイル',
   '従来 端末 グループ 戻し 指定 アプリ 再 インストール',
   'グループ から 異常 端末 グループ エンド ポイント 自動',
   '異常 端末 グループ エンド ポイント 自動 移動 隔離',
   '実施 異常 端末 グループ 構成 プロファイル アプリ 割り当て',
   'ポイント 所属 部門 対応 た 端末 グループ から']],
 [6,
  ['移動 端末 グループ によって 適用 構成 プロファイル 詳細',
   'データ 値 IP アドレス によって 所属 端末 グループ',
   'グループ 私物 端末 グループ 作る 考え られ により',
   'ので 端末 移動 端末 グループ 設定 値 従い',
   '設定 異なる ので 端末 移動 端末 グループ 設定',
   'IP アドレス によって 所属 端末 グループ 移動 端末']],
 [7,
  ['Windows Configuration Designer 作成 プロビジョニングパッケージ MobiControl 端末 グループ',
   'ユーザ 間 端末 共用 G 異なる 端末 グループ',
   'グループ 端末 全部 働きかけ しか つ 端末 グループ',
   '表示 たい として 端末 グループ あて メッセージ 送る',
   'イン ポート 所属 端末 グループ 再 配置 可能',
   'なる 表示 同じ 端末 グループ 異なる 端末 OS',
   '月日 MobiControl v v ハイライト 異なる 端末 グループ',
   '全般 端末 所属 端末 グループ に関する CSV ファイル',
   '従って つ 端末 グループ 全 端末 働きかけ 方式',
   'つ 作成 G 異なる 端末 グループ ユーザ 間',
   'まし た 自動 DEP 登録 端末 端末 グループ',
   '端末 共用 端末 グループ 異なる 構成 プロファイ